In [ ]:
%load_ext autoreload
%autoreload 2
%run A_simple_net.ipynb

In [ ]:
%mkdir -p outputs

In [ ]:
%mkdir -p Results/20201204/

# scanning over single parameters

## initializing parameters

In [ ]:
tag = '2020-12-04_scan_' + simulator

In [ ]:
buildCPUTime, simCPUTime, writeCPUTime = run(verbose=True)

## Running Model and Loading Results

In [ ]:
import os
import pandas as pd
verbose = True
def scan(variable, values, tag=tag, verbose=verbose):
    filename = 'outputs/{tag}_{variable}.json'.format(tag=tag, variable=variable)
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=[variable, 'buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, value in enumerate(values):
            if verbose: print('{variable=}---{value=}'.format(value=value, variable=variable))
            args = {}
            args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= {variable: value, 'buildCPUTime':buildCPUTime, 'simCPUTime':simCPUTime, 'writeCPUTime':writeCPUTime}
        df.to_json(filename)
    return df

In [ ]:
import matplotlib.pyplot as plt

# https://stackoverflow.com/questions/31978948/python-stats-models-quadratic-term-in-regression
def plot(df, do_intercept=False):
    variable = df.columns[0]
    fig, ax = plt.subplots(figsize=(8,8))
    
    for regressor, color in zip(['buildCPUTime', 'simCPUTime', 'writeCPUTime'],
                                ['r', 'g', 'b']):

        import patsy
        if do_intercept:
            y, X = patsy.dmatrices('{regressor} ~ {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')
        else:
            y, X = patsy.dmatrices('{regressor} ~ 0 + {variable}'.format(regressor=regressor, variable=variable), data=df, return_type='dataframe')

        import statsmodels.api as sm
        fit = sm.OLS(y, X).fit()
        print(fit.summary())

        if do_intercept: intercept = fit.params.Intercept
        slope = fit.params[variable]

        values = np.array(df[variable])
        if do_intercept:
            ax.plot(values, intercept + slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable} + {intercept:.3f}")
        else:
            ax.plot(values, slope*values, c=color, lw=2, label=f"{regressor}={slope:.3f}*{variable}")
        df.plot(x=variable, y=regressor, c=color, lw=1, ls='-.', ax=ax, legend=False)

    ax.set_xlim(0)
    ax.set_ylabel('Time (ms)')
    ax.set_ylim(0)
    ax.legend()
    return fig, ax

## plot as a function of neuron numbers

In [ ]:
import numpy as np
N_pops = np.array([int(k)**2 for k in np.linspace(8**2, 45**2, 14, endpoint=True)**.5])
N_pops

In [ ]:
results_pop = scan('N_pop', N_pops)

In [ ]:
fig, ax = plot(results_pop)

In [ ]:
fig.savefig(tag + '_N_pop.png')

## plot as a function of simtime

In [ ]:
simtimes = np.linspace(500, 3000, 15, endpoint=True)
simtimes

In [ ]:
results_simtime = scan('simtime', simtimes)

In [ ]:
fig, ax = plot(results_simtime, do_intercept=True)      

In [ ]:
fig.savefig(tag + '_simtime.png')

# scanning over two parameters

## Setting-up model and initializing parameters

In [ ]:
tag = '2020-12-04_scan-2D_' + simulator

## Running Model and Loading Results

In [ ]:
variables = ['N_pop', 'simtime']
filename = 'outputs/' + tag
for variable in variables: filename +=  '_' + variable
filename += '.json'

In [ ]:
import os
import time
import pandas as pd
verbose = True
def scan(variables, values, tag=tag, verbose=verbose):
    filename = 'outputs/' + tag
    for variable in variables: filename +=  '_' + variable
    filename += '.json'
    try:
        df = pd.read_json(filename)
        print('Loaded file')
    except:
        df = pd.DataFrame([], columns=variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'])
        for i, values_ in enumerate(values):
            if verbose: print('{variable}---{values_}'.format(values_=values_, variable=variable))
            args = {}
            for variable, value in zip(variables, values_):
                args[variable] = value
            buildCPUTime, simCPUTime, writeCPUTime = run(**args)
            df.loc[i]= dict(zip(variables + ['buildCPUTime', 'simCPUTime', 'writeCPUTime'], 
                                list(values_) + [buildCPUTime, simCPUTime, writeCPUTime]))

        df.to_json(filename)
        
    return df

## Running time as a function of neuron numbers and simtime

In [ ]:
import numpy as np

In [ ]:
N_trials = 200
N_pops = [int(k)**2 for k in np.linspace(15, 60, N_trials, endpoint=True)]
simtimes = np.linspace(200, 3000, N_trials, endpoint=True)

In [ ]:
np.random.seed(42)
np.random.shuffle(N_pops)
np.random.shuffle(simtimes)
values = zip(N_pops, simtimes)

In [ ]:
results = scan(variables=variables, values=values)
results

## analysis

In [ ]:
results_ext = results.copy()
results_ext['simtime_N_pop'] = results_ext['simtime'] * results_ext['N_pop']
results_ext

In [ ]:
variables_ext = ['N_pop', 'simtime_N_pop']
X = results_ext[variables_ext].values.reshape(-1, len(variables_ext))


https://aegis4048.github.io/mutiple_linear_regression_and_visualization_in_python

See https://patsy.readthedocs.io/en/latest/quickstart.html

In [ ]:
import patsy
import statsmodels.api as sm

for regressor in ['buildCPUTime', 'simCPUTime', 'writeCPUTime']:
    y = results_ext[regressor].values
    #print(variables_ext, X.shape, y.shape)
    y, X = patsy.dmatrices('{regressor} ~ 0 + N_pop + simtime_N_pop'.format(regressor=regressor), data=results_ext, return_type='dataframe')

    fit = sm.OLS(y, X).fit()
    fit.summary()

    print('{regressor} (ms) = {fit0:.3f} * N_pop + {fit1:.3f}/1000 * simtime (ms) * N_pop'.format(regressor=regressor, fit0=fit.params[0], fit1=fit.params[1]*1000))